In [8]:
import os
import pandas as pd
import tensorflow as tf 
import numpy as np
import glob
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import classification_report


if os.getcwd().split('/')[-1] == "evaluation":
    %cd ../..

# Import noise simulator
from scripts.utils import read_dcm

In [9]:
def get_train_and_test_data():
  df=pd.read_csv("data/RSNA_BME548/data.csv")

  files=glob.glob("data/RSNA_BME548/images/*.dcm")

  label=[]
  img_array=[]
  for i in files:
    filename = os.path.basename(i)
    img=read_dcm(i)
    img_array.append(np.array(img))
    # Get value of column "any" for the row with filename as index
    label.append(df.loc[df['filename'] == filename]['any'].values[0])

  x = np.array(img_array)
  x = x/255
  y = np.array(label)

  x = np.expand_dims(x,3)
  X_train, X_test, y_train, y_test = train_test_split(x,y, test_size = 0.15, random_state = 0)
  
  return X_train, X_test, y_train, y_test

In [10]:
class Block(tf.keras.Model):
    def __init__(self, filters, kernel_size, repetitions, pool_size=2, strides=2):
        super(Block, self).__init__()
        self.filters =filters
        self.kernel_size = kernel_size
        self.repetitions = repetitions
        
        # Define a conv2D_0, conv2D_1, etc based on the number of repetitions
        for i in range(self.repetitions):
            
            # Define a Conv2D layer, specifying filters, kernel_size, activation and padding.
            vars(self)[f'conv2D_{i}'] = tf.keras.layers.Conv2D(self.filters,self.kernel_size,activation='relu',padding="same")
        
        # Define the max pool layer that will be added after the Conv2D blocks
        self.max_pool = tf.keras.layers.MaxPooling2D(pool_size, strides=strides)
  
    def call(self, inputs):
        # access the class's conv2D_0 layer
        conv2D_0 = vars(self)['conv2D_0']
        
        # Connect the conv2D_0 layer to inputs
        x = conv2D_0(inputs)

        # for the remaining conv2D_i layers from 1 to `repetitions` they will be connected to the previous layer
        for i in range(1,self.repetitions):
            # access conv2D_i by formatting the integer `i`. (hint: check how these were saved using `vars()` earlier)
            conv2D_i = vars(self)[f'conv2D_{i}']
            
            # Use the conv2D_i and connect it to the previous layer
            x = conv2D_i(x)

        # Finally, add the max_pool layer
        max_pool = self.max_pool(x)
        
        return max_pool

In [11]:
class MyVGG(tf.keras.Model):

    def __init__(self, num_classes):
        super(MyVGG, self).__init__()

        # Creating blocks of VGG with the following 
        # (filters, kernel_size, repetitions) configurations
        self.block_a = Block(filters=64, kernel_size=3, repetitions=2)
        self.block_b = Block(filters=128, kernel_size=3, repetitions=2)
        self.block_c = Block(filters=256, kernel_size=3, repetitions=3)
        self.block_d = Block(filters=512, kernel_size=3, repetitions=3)
        self.block_e = Block(filters=512, kernel_size=3, repetitions=3)

        # Classification head
        # Define a Flatten layer
        self.flatten = tf.keras.layers.Flatten()
        # Create a Dense layer with 256 units and ReLU as the activation function
        self.fc = tf.keras.layers.Dense(256,activation='relu')
        # Finally add the softmax classifier using a Dense layer
        self.classifier = tf.keras.layers.Dense(num_classes, activation='softmax')

    def call(self, inputs):
        # Chain all the layers one after the other
        x = self.block_a(inputs)
        x = self.block_b(x)
        x = self.block_c(x)
        x = self.block_d(x)
        x = self.block_e(x)
        x = self.flatten(x)
        x = self.fc(x)
        x = self.classifier(x)
        return x

In [12]:
def get_model_accuracy(X_train, X_test, y_train, y_test):
  vgg = MyVGG(num_classes=2)
  vgg.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
  vgg.fit(X_train,y_train,epochs=5)

  # vgg.save_weights("models/vgg.weights.h5")

  y_pred=vgg.predict(X_test)

  y_pred_binary=[]
  for a in y_pred:
    y_pred_binary.append(np.argmax(a))

  y_pred_binary=np.array(y_pred_binary)

  accuracy = metrics.accuracy_score(y_test,y_pred_binary)

  print("Accuracy of model=",accuracy)

  print(classification_report(y_test, y_pred_binary, target_names=['0','1']))

  return accuracy

In [13]:
X_train, X_test, y_train, y_test = get_train_and_test_data()

In [ ]:
get_model_accuracy(X_train, X_test, y_train, y_test)